<a href="https://colab.research.google.com/github/misterbwang/Comic-AudioBooks/blob/master/keras_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3733, done.
remote: Counting objects: 100% (3733/3733), done.
remote: Compressing objects: 100% (2863/2863), done.
remote: Total 3733 (delta 1063), reused 1832 (delta 820), pack-reused 0
Receiving objects: 100% (3733/3733), 48.75 MiB | 6.83 MiB/s, done.
Resolving deltas: 100% (1063/1063), done.
Updating files: 100% (3393/3393), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 16.5 MB/s eta 0:00

In [ ]:
!pip uninstall numpy
!pip install numpy==1.21.1
!pip install tensorflow==2.11.0
!pip install easyocr


Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.9
    /usr/local/lib/python3.9/dist-packages/numpy-1.23.5.dist-info/*
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libopenblas64_p-r0-742d56dc.3.20.so
    /usr/local/lib/python3.9/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.9/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.23.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 47.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import easyocr

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
#from object_detection.builders import model_builder

/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_18SourceLocationImplE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN3tsl13Gcs

In [ ]:
tf.__version__

'2.11.0'

In [ ]:
np.__version__

'1.22.4'

In [ ]:
%mkdir /content/test/
%cd /content/test/

/content/test


In [ ]:
!curl -L "https://app.roboflow.com/ds/alCHfcc8BR?key=MACtAQCNlj" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   892  100   892    0     0   2505      0 --:--:-- --:--:-- --:--:--  2505
100 50.3M  100 50.3M    0     0  39.4M      0  0:00:01  0:00:01 --:--:--  118M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/0013_jpg.rf.218cb1451c4fcc1adf66d16bf67ebb26.jpg  
 extracting: test/0013_jpg.rf.b589bb70c5a6dc8c7c595302f7f8d113.jpg  
 extracting: test/0020_jpg.rf.62fa4d59e19a29ab41ee67006cd9e3dc.jpg  
 extracting: test/0024_jpg.rf.fdd5fbaf4ac7521f5a265adb1be8c678.jpg  
 extracting: test/0035_jpg.rf.043de8a9a55ce4859037feb95fbb3551.jpg  
 extracting: test/0044_jpg.rf.94ce146b22c5d4d05f2d7bb9d365e4e9.jpg  
 extracting: test/0056_jpg.rf.6735b426b3bc27300a0ef3a930129896.jpg  
 extracting: test/0061_jpg.rf.71326a7500202e6a727abed2d3e390c9.jpg  
 extracting: t

In [ ]:
TEST_IMAGE_PATHS = glob.glob('/content/test/test/*.jpg')
image_path = random.choice(TEST_IMAGE_PATHS)

In [ ]:
TEST_IMAGE_PATHS[6]

'/content/test/test/Amazing-Spider-Man-010-020_jpg.rf.fc16e76bea0fe8ec58aa1c3e4b7e0a16.jpg'

In [ ]:
image.load_img(TEST_IMAGE_PATHS[6],target_size=(680,680))

NameError: ignored

In [ ]:
!pip install pytesseract
!apt install tesseract-ocr
!pip install easyocr
!pip install -U git+https://github.com/faustomorales/keras-ocr.git#egg=keras-ocr
!pip install keras-ocr

In [ ]:
#import pkg_resources
import pytesseract
import cv2
#import argparse

from PIL import Image

from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing import image

import zipfile
import datetime
import string
import glob
import math
import os

import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

import keras_ocr

#assert tf.test.is_gpu_available()

In [ ]:
try:
  from google.colab import drive
  import os
  drive.mount('/content/drive')
  data_dir = 'drive/My Drive/colab/keras-ocr'
  os.makedirs(data_dir, exist_ok=True)
except ImportError:
  data_dir = '.'

In [ ]:
alphabet = string.digits + string.ascii_letters + '!?. '
recognizer_alphabet = ''.join(sorted(set(alphabet.lower())))
fonts = keras_ocr.data_generation.get_fonts(
    alphabet=alphabet,
    cache_dir=data_dir
)
backgrounds = keras_ocr.data_generation.get_backgrounds(cache_dir=data_dir)

In [ ]:
detector = keras_ocr.detection.Detector(weights='clovaai_general')
recognizer = keras_ocr.recognition.Recognizer(
    alphabet=recognizer_alphabet,
    weights='kurapan'
)
recognizer.compile()
for layer in recognizer.backbone.layers:
    layer.trainable = False

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline(detector=detector, recognizer=recognizer)

In [ ]:
test_images = TEST_IMAGE_PATHS[:1]

In [ ]:
test_images

In [ ]:
img = image.load_img(TEST_IMAGE_PATHS[7],target_size=(680,680)) #1080, 880
img = image.img_to_array(img, dtype='uint8')

cv2_imshow(img)

In [ ]:
reader = easyocr.Reader(['en'], gpu=True)

In [ ]:
reader.readtext(img, decoder = 'wordbeamsearch', paragraph = True, low_text=.15)